In [1]:
import numpy as np
%matplotlib notebook
import sys
sys.path.append('..')

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D 
from utils.data import fetch_data, fetch_data_with_label, fetch_data_with_label_per_step, generate_data, label_to_int, divide_data
from utils.plotting import plot
from utils.clustering import agglomerative, kmeans, gaussian_mixture, Clustering
from sklearn.metrics import adjusted_rand_score
import itertools
import pickle


In [2]:
import glob

target_dir = '../../roboSim/data/bin_picking/*/*.pickle'
# names = [name for name in listdir(target_dir) if name != 'archive']
# filenames = [target]

logfile_path = glob.glob(target_dir)
seqs, label = fetch_data_with_label_per_step(logfile_path, n_states = 2)

In [16]:
import numpy as np
import sys
sys.path.append('..')

from utils.data import fetch_data_with_label_per_step, label_to_int, balance_data
from utils.clustering import Clustering
from utils.encoding import make_good_unitary, encode_feature
from sklearn.metrics import adjusted_rand_score
import itertools
import argparse
import glob

'''
run this file : python ./encoding_3d.py -t assembly -al gaussian -an True -af p_norm, euclidean, euclidean
'''

def process_data(selected_data, selected_label, with_anchor, task = 'assembly'):
    '''
    This method process the 4-d data(3-d coordinates + 1-d object kind) by:
     1. Balancing the data
     2. Add anchor object information if needed
     3. Change the index encoding(0 for nut, 1 for bolt) to distributed encodeing(ex. [0.8, 0.2] for nut and [0.9, 0.1] for bolt)
    '''
    u_coord = 0  # The average shift between the approximated coordinates and ground truth
    sigma_coord = 0.006
    u_kind = 0.1
    sigma_kind = 0.05
    u_anchor = 0.1
    sigma_anchor= 0.05
    
    ind_not_noisy = selected_label != 'Noise_data'
    data = selected_data[ind_not_noisy, :]
    label = selected_label[ind_not_noisy]
    
    selected_data_balanced, selected_label_balanced = balance_data(data, label)
    selected_label_int = label_to_int(selected_label_balanced)

    # Object coordinates information
    coord = selected_data_balanced[:,0:3]
    obj_kind = selected_data_balanced[:, 3]
    noise = np.random.normal(u_coord, sigma_coord, coord.shape)
    coord_noisy = coord + noise

    # Object kind information
    ind_nut = np.where(obj_kind == 0)[0]
    ind_bolt = np.where(obj_kind == 1)[0]
    n_data = coord.shape[0]
    one_hot_kind = np.zeros((n_data,2))
    one_hot_kind[np.arange(obj_kind.size),obj_kind.astype(int)] = 1
    noise_kind = np.random.normal(u_kind, sigma_kind, obj_kind.size)
    kind_noisy = abs(one_hot_kind - np.column_stack((noise_kind, noise_kind)))

    if task == 'assembly':
        # Anchor object information
        ind_table = np.concatenate((np.where(selected_label_balanced == 'Nut on table')[0], np.where(selected_label_balanced == 'Bolt on table')[0]))
        ind_jig = np.delete(np.arange(n_data), ind_table)
        one_hot_anchor = np.zeros((n_data,2))
        one_hot_anchor[ind_table,0] = 1
        one_hot_anchor[ind_jig,1] = 1
        noise_anchor = np.random.normal(u_anchor, sigma_anchor, obj_kind.size)
        anchor_noisy = abs(one_hot_anchor - np.column_stack((noise_anchor, noise_anchor)))

    elif taks == 'bin_picking':
        ind_bin_origin = np.where(selected_label_balanced == 'Object in origin bin')[0]
        ind_bin_target = np.delete(np.arange(n_data), ind_table)
        one_hot_anchor = np.zeros((n_data,2))
        one_hot_anchor[ind_bin_origin,0] = 1
        one_hot_anchor[ind_bin_target,1] = 1
        noise_anchor = np.random.normal(u_anchor, sigma_anchor, obj_kind.size)
        anchor_noisy = abs(one_hot_anchor - np.column_stack((noise_anchor, noise_anchor)))
    if with_anchor:
        data_concat = [coord_noisy, kind_noisy, anchor_noisy]
    else:
        data_concat = [coord_noisy, kind_noisy]
    return data_concat, selected_label_int

def test_3d(task, algorithm, with_anchor, affinity):
    if task == 'assembly':
        n_states = 4
        n_clusters = 5
        target_dir = '../../roboSim/data/assembly/*/*.pickle'
    elif task == 'bin_picking':
        n_states = 2
        n_clusters = 2
        target_dir = '../../roboSim/data/bin_picking/*/*.pickle'
    logfile_path = glob.glob(target_dir)
    #coordinates and object kind for nut and bolt
    data, label = fetch_data_with_label_per_step(logfile_path, n_states)
    for i,x in enumerate(data):
        # Get rid of the data that only has one datapoint
        datapoint = np.array(list(itertools.chain.from_iterable(data[i])))
        if datapoint.shape[0] < 2:
            data.pop(i)
            label.pop(i)
    n_test = 100
    result = []
    rand_score = 0
    n_success = 0
    for i in range(n_test):
        ind = np.random.choice(len(data))
        selected_data = np.array(list(itertools.chain.from_iterable(data[ind])))
        selected_label = np.array(list(itertools.chain.from_iterable(label[ind])))
        data_concat, selected_label_int = process_data(selected_data, selected_label, with_anchor, task = 'assembly')
        if with_anchor:
            affinity = affinity
        else:
            affinity = affinity[:2]
        C = Clustering(data_concat, algorithm, affinity, n_clusters, thres = None, p = -2)
        estimated_label = C.predict()
        n_original = selected_data.shape[0]
        print(n_original, estimated_label[:n_original], selected_label_int, len(selected_label), ind)
        temp = adjusted_rand_score(estimated_label[:n_original], selected_label_int[:n_original])
        rand_score += temp
        if temp == 1:
            n_success +=1
        average_rand = rand_score / n_test
        success_rate = n_success / n_test * 100
    print(f'Algorithm is {algorithm}, Affinity is {affinity}, With anchor is {with_anchor}. \n \
            The average rand score is {average_rand}.\n \
            The success rate is {success_rate}%')
    return average_rand, success_rate

    


In [6]:
test_3d('bin_picking', 'gaussian', False, 'euclidean')

8 [1 0 1 1 0 0 1 1] [1 1 1 1 0 1 1 1 0 0 0 0 0 0] 8 200
4 [0 1 0 1] [1 1 0 1 0 0] 4 497
4 [0 0 0 1] [1 1 1 0 0 0] 4 190
4 [0 0 0 1] [1 1 1 0 0 0] 4 156
4 [0 1 0 1] [1 1 0 1 0 0] 4 182
6 [1 1 1 1 0 1] [1 1 1 1 0 1 0 0 0 0] 6 92
8 [0 1 1 1 0 0 1 1] [1 1 1 1 0 1 1 1 0 0 0 0 0 0] 8 346
8 [0 1 1 1 0 1 1 1] [1 1 1 1 0 1 1 1 0 0 0 0 0 0] 8 121
4 [0 1 0 1] [1 1 0 1 0 0] 4 490
8 [1 1 0 0 1 1 0 0] [1 1 1 1 1 1 0 1 0 0 0 0 0 0] 8 457
8 [0 0 0 0 0 0 1 0] [1 1 1 1 1 1 0 1 0 0 0 0 0 0] 8 83
4 [1 1 0 1] [1 1 0 1 0 0] 4 341
6 [1 1 1 0 1 1] [1 1 1 0 1 1 0 0 0 0] 6 464
8 [0 0 0 0 0 0 1 0] [1 1 1 1 1 1 0 1 0 0 0 0 0 0] 8 501
6 [0 0 0 1 0 0] [1 1 1 0 1 1 0 0 0 0] 6 474
4 [1 1 0 1] [1 1 0 1 0 0] 4 126
8 [0 0 0 0 0 0 1 0] [1 1 1 1 1 1 0 1 0 0 0 0 0 0] 8 405
8 [1 1 0 0 1 0 0 0] [1 1 1 1 0 1 1 1 0 0 0 0 0 0] 8 358
4 [0 1 0 1] [1 1 0 1 0 0] 4 8


IndexError: too many indices for array

In [18]:
task = 'bin_picking'
algorithm = 'gaussian'
affinity = 'euclidean'
with_anchor = True

if task == 'assembly':
    n_states = 4
    n_clusters = 5
    target_dir = '../../roboSim/data/assembly/*/*.pickle'
elif task == 'bin_picking':
    n_states = 2
    n_clusters = 2
    target_dir = '../../roboSim/data/bin_picking/*/*.pickle'
logfile_path = glob.glob(target_dir)
#coordinates and object kind for nut and bolt
data, label = fetch_data_with_label_per_step(logfile_path, n_states)

n_test = 100
result = []
rand_score = 0
n_success = 0
n_skip = 0
for i in range(n_test):
    ind = np.random.choice(len(data))
    selected_data = np.array(list(itertools.chain.from_iterable(data[ind])))
    selected_label = np.array(list(itertools.chain.from_iterable(label[ind])))
    if selected_data.shape[0] < 2:
        # Skip cases when there is only one datapoint
        n_skip += 1
        continue
    data_concat, selected_label_int = process_data(selected_data, selected_label, with_anchor, task = 'assembly')
    if with_anchor:
        affinity = affinity
    else:
        affinity = affinity[:2]
    C = Clustering(data_concat, algorithm, affinity, n_clusters, thres = None, p = -2)
    estimated_label = C.predict()
    n_original = selected_data.shape[0]
    print(n_original, estimated_label[:n_original], selected_label_int, len(selected_label), ind)
    temp = adjusted_rand_score(estimated_label[:n_original], selected_label_int[:n_original])
    rand_score += temp
    if temp == 1:
        n_success +=1
    average_rand = rand_score / n_test
    success_rate = n_success / n_test * 100
print(f'Algorithm is {algorithm}, Affinity is {affinity}, With anchor is {with_anchor}. \n \
        The average rand score is {average_rand}.\n \
        The success rate is {success_rate}% \n\
        The number of test is {n_test - n_skip}')


    


4 [1 0 1 0] [1 1 0 1 0 0] 4 528
8 [0 0 0 1 0 0 0 1] [2 2 2 1 0 2 2 1 0 0 0 0 1 1 1] 8 368
8 [0 0 0 0 0 1 1 0] [1 1 1 1 1 0 0 1 0 0 0 0] 8 391
4 [0 1 0 1] [1 1 1 0 0 0] 4 167
4 [1 0] [1 0] 4 336
8 [0 0 0 0 1 0 0 0] [1 1 1 1 0 1 1 1 0 0 0 0 0 0] 8 424
4 [1 0 1 0] [1 1 0 1 0 0] 4 152
8 [1 1 0 0 1 1 0 0] [1 1 1 1 1 1 0 1 0 0 0 0 0 0] 8 412
4 [0 1] [0 0] 4 288
4 [0 1] [0 0] 4 302
8 [1 1 1 1 0 1 1 1] [1 1 1 1 0 1 0 1 0 0 0 0] 8 499
8 [0 0 1 1 0 0 1 1] [1 1 1 1 0 1 1 1 0 0 0 0 0 0] 8 206
6 [1 1 1 1 0 1] [1 1 1 1 0 1 0 0 0 0] 6 217
4 [0 0 1 0] [1 1 0 1 0 0] 4 148
4 [1 0 1 0] [1 1 1 0 0 0] 4 7
8 [0 0 1 1 0 0 1 1] [1 1 1 1 1 0 0 1 0 0 0 0] 8 86
4 [1 0] [0 0] 4 300
6 [1 1 0 1 0 0] [1 1 0 1 0 0] 6 422
4 [1 0] [0 0] 4 316
4 [0 1 0 1] [1 1 1 0 0 0] 4 21
4 [0 0 0 1] [1 1 1 0 0 0] 4 46
4 [1 1 1 0] [1 1 1 0 0 0] 4 162
8 [1 1 0 0 1 1 0 0] [1 1 1 1 0 1 1 1 0 0 0 0 0 0] 8 393
8 [0 0 0 0 1 0 0 0] [1 1 1 1 0 1 1 1 0 0 0 0 0 0] 8 397
4 [0 1] [1 0] 4 426
4 [0 1 0 1] [1 1 1 0 0 0] 4 192
12 [1 1 1 0 0 0 1 1 1 0

In [11]:
print(ind)

226


In [13]:
data.shape

AttributeError: 'list' object has no attribute 'shape'

In [15]:
selected_data

array([[ 0.40663362,  0.05692914, -0.16544304,  1.        ],
       [ 0.40147898,  0.05462287, -0.07742524,  1.        ]])

In [17]:
selected_label

array(['Object in origin bin', 'Object in origin bin',
       'Object in target bin', 'Object in origin bin'], dtype='<U20')

In [10]:
selected_label

array(['Object in origin bin', 'Object in origin bin',
       'Object in target bin', 'Object in origin bin'], dtype='<U20')

In [34]:
selected_data.shape[0]

1

In [19]:
len(selected_label)

1

In [12]:
selected_data

array([[ 0.43498053,  0.03105522, -0.13032542,  0.        ],
       [ 1.28317173,  0.02727028, -0.2515229 ,  1.        ],
       [-0.50452024,  0.03427605, -0.47142175,  0.        ],
       [ 1.28314302,  0.02715544, -0.25391189,  1.        ]])

In [13]:
selected_label

array(['Object in origin bin', 'Noise data', 'Object in target bin',
       'Noise data'], dtype='<U20')

In [5]:
a = [0,1,2,3]

In [6]:
a.pop([0,1])

TypeError: 'list' object cannot be interpreted as an integer